In [1]:
!pip install import_ipynb
!pip install bayesian-optimization
import import_ipynb
from google.colab import drive
import os
import pandas as pd
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/Intro to DS - Capstone/Code/src')
from utilities import *

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=1b59a920057a7e783d3af984769855af7c14c6380081d7e312a89a06729f7ffe
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=81929879186c445c90152a67e87807743b31cfd3b0e4b19c1c3d6234eec2bcdd
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization
Mounted at /content/gdrive


In [2]:
# Imports
import numpy as np
import pandas as pd
import os
import pathlib
from tqdm import tqdm as tqdm
import time
import pickle
from datetime import date
import json
import urllib.request

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score



!pip install geopandas 
import geopandas as gpd
import folium
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)


# Define global variables
base_path = '/content/gdrive/MyDrive/Intro to DS - Capstone/'
raw_path = base_path + 'Data/Raw/'
interim_path = base_path + 'Data/Interim/'
results_path = base_path + 'Data/Results/'

     |████████████████████████████████| 1.0 MB 5.7 MB/s 
     |████████████████████████████████| 15.4 MB 91 kB/s 
     |████████████████████████████████| 6.3 MB 11.2 MB/s 


# 0. Load necessary data

In [3]:
from scipy.stats import f_oneway
from scipy import stats
from scipy.stats import chisquare

data = pd.read_excel(raw_path + 'default of credit card clients.xlsx', header = 1)
data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


# 1 Hypothesis testing

### 1.1 Effect of education on creditworthiness

In [4]:
chisquare(f_obs = [data[data.EDUCATION == i]['default payment next month'].sum() for i in [1,2,3]],
          f_exp = [int(data[data.EDUCATION == i].shape[0]*data['default payment next month'].mean()) for i in [1,2,3]])

Power_divergenceResult(statistic=77.04265134920404, pvalue=1.8638065082655474e-17)

### 1.2 Effect of age on creditworthiness

In [5]:
def age_bins(x):
  if (x<25): return '1. < 25'
  if (x<35): return '2. 25-35'
  if (x<50): return '3. 36-50'
  if (x<65): return '4. 50-65'
  return '5. >65'

data['age_bucket'] = data.AGE.apply(lambda x: age_bins(x))

chisquare(f_obs = [data[data.age_bucket == i]['default payment next month'].sum() for i in data.age_bucket.unique()],
          f_exp = [int(data[data.age_bucket == i].shape[0]*data['default payment next month'].mean()) for i in data.age_bucket.unique()])

Power_divergenceResult(statistic=63.77187272273111, pvalue=4.667926401598622e-13)

### 1.3 Effect of credit limit on default

In [8]:
print(data['LIMIT_BAL'].median())
chisquare(f_obs = [data[data.LIMIT_BAL < 140000]['default payment next month'].sum(), data[data.LIMIT_BAL >= 140000]['default payment next month'].sum()],
          f_exp = [int(data[data.LIMIT_BAL < 140000].shape[0] * data['default payment next month'].mean()), int(data[data.LIMIT_BAL >= 140000].shape[0] * data['default payment next month'].mean())])

140000.0


Power_divergenceResult(statistic=528.1140204214378, pvalue=7.265328867643383e-117)